In [93]:
import os
print(os.getcwd())

/home/hiimbias/Workspace/aiagent


In [94]:
os.chdir('/home/hiimbias/Workspace/aiagent')
print(os.getcwd())  # Verify the change

/home/hiimbias/Workspace/aiagent


In [95]:
from modules.openai.openai_executor import OpenAIExecutor
from core.utils.tool_wrapper import tool
from pydantic import BaseModel
from typing import Annotated
import openai
import os
from modules.tools.get_weather import get_weather_tool

In [96]:
@tool(
    description="Calculate the sum of 2 numbers", # Define the tool description
)
def calculate(a: Annotated[int, "a"], b: Annotated[int, "b"]) -> int:
    """Calculate the sum of two numbers"""
    return a + b

In [97]:
calculate.schema

{'type': 'function',
 'function': {'name': 'calculate',
  'description': 'Calculate the sum of two numbers',
  'strict': True,
  'parameters': {'properties': {'a': {'title': 'A', 'type': 'integer'},
    'b': {'title': 'B', 'type': 'integer'}},
   'required': ['a', 'b'],
   'type': 'object',
   'additionalProperties': False}}}

In [98]:
# Initialize OpenAI client
client = openai.OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

# Initialize LLM service
executor = OpenAIExecutor(
    client=client,
    model="gpt-4o-mini",
    system_message="""
    You are a helpful assistant that can answer questions and help with tasks.
    You are also able to use tools to get information.
    """,
    tools=[get_weather_tool, calculate],
    temperature=0.5,
    max_tokens=100,
    top_p=1.0,
)

generator = executor.execute(
    messages=[
        {"role": "user", "content": "What's the weather like in New York?"}
    ]
)

for response in generator:
    print(response)

ValueError: llm_provider must be provided

In [ ]:
executor.get_history()

[{'role': 'system',
  'content': '\n    You are a helpful assistant that can answer questions and help with tasks.\n    You are also able to use tools to get information.\n    \n        Current date and time: 2025-05-14 09:30:38\n\n        \n        '}]

In [ ]:
executor.clear_history()

[{'role': 'system',
  'content': '\n    You are a helpful assistant that can answer questions and help with tasks.\n    You are also able to use tools to get information.\n    \n        Current date and time: 2025-05-14 09:30:38\n\n        \n        '}]

In [ ]:
while True:
    user_input = input("User: ")
    print(f"User: {user_input}")
    
    if user_input.lower() == "exit":
        break

    generator = executor.execute(
        messages=[
            {"role": "user", "content": user_input}
        ]
    )

    for response in generator:
        if response.content is not None:
            print(f"AI: {response.content}")
        if response.tool_calls:
            [print(f"Tool Calls: {tool_call}") for tool_call in response.tool_calls]

User: Hello
AI: Hello! How can I assist you today?


KeyboardInterrupt: Interrupted by user

In [ ]:
executor.get_history()

[{'role': 'system',
  'content': '\n    You are a helpful assistant that can answer questions and help with tasks.\n    You are also able to use tools to get information.\n    \n        Current date and time: 2025-05-03 00:00:28\n\n        \n        '}]